In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("Churn_Modelling.csv")
df.head()


In [ ]:
df.dtypes

In [ ]:
df.drop(["RowNumber", "CustomerId", "Surname"], axis=1, inplace=True)
df

In [ ]:
# change (male, female) to (1,0) using Label Encoder
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['Gender'] = label_encoder.fit_transform(df['Gender'])
df


In [ ]:
# encode "Geography" using OneHotEncoder

from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder()

encode_geog_col = pd.DataFrame(one_hot_encoder.fit_transform(df[['Geography']]).toarray())
df = df.join(encode_geog_col)
df.drop('Geography', axis=1, inplace=True)


In [ ]:
df

In [ ]:
# scale this shit

from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()

features = df.copy().columns.values
min_max_scaler.fit(df)
df = pd.DataFrame(min_max_scaler.fit_transform(df))
df.columns = features
df

In [ ]:
# train, test, split
from sklearn.model_selection import train_test_split

X = df.drop('Exited', axis=1)
y = df['Exited']

X_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25)
X_train

In [ ]:
num_inputs = len(X_train.columns.values)
num_inputs

In [ ]:
# train!
import tensorflow as tf
from tensorflow import keras

model = keras.Sequential([
    keras.layers.Dense(num_inputs, input_shape=(num_inputs,), activation="relu"),
    keras.layers.Dense(num_inputs, activation="relu"),
    keras.layers.Dense(num_inputs, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid"),
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


In [ ]:
model.fit(X_train, y_train, epochs=100)

In [ ]:
y_predict = model.predict(x_test)
y_predict = (y_predict > 0.5).astype(int)

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predict)
cm